#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/t/ab8726f0cfc84544abbae69a6be88071

## Librerias

In [1]:
import pandas as pd
import numpy as np
import urllib.request
from PIL import Image
import re
import matplotlib.pyplot as plt
import seaborn as sns

## Data

In [16]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
train = pd.read_csv(r"C:\Users\Joan Oliver\TheBridge-DataScienceBootcamp\data-science-pt-2023-01\Machine_Learning\1-Supervisado\1-Regression\Kaggle1\data\train.csv")
test = pd.read_csv(r"C:\Users\Joan Oliver\TheBridge-DataScienceBootcamp\data-science-pt-2023-01\Machine_Learning\1-Supervisado\1-Regression\Kaggle1\data\test.csv")
sample_submission = pd.read_csv(r"C:\Users\Joan Oliver\TheBridge-DataScienceBootcamp\data-science-pt-2023-01\Machine_Learning\1-Supervisado\1-Regression\Kaggle1\data\sample_submission.csv")


## Data exploration

In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         912 non-null    int64  
 1   Company           912 non-null    object 
 2   Product           912 non-null    object 
 3   TypeName          912 non-null    object 
 4   Inches            912 non-null    float64
 5   ScreenResolution  912 non-null    object 
 6   Cpu               912 non-null    object 
 7   Ram               912 non-null    object 
 8   Memory            912 non-null    object 
 9   Gpu               912 non-null    object 
 10  OpSys             912 non-null    object 
 11  Weight            912 non-null    object 
 12  Price_euros       912 non-null    float64
dtypes: float64(2), int64(1), object(10)
memory usage: 92.8+ KB


## Data processing

In [18]:
data_df = train.copy()
data_df.drop(["laptop_ID", "Product", "Weight"], axis=1, inplace=True)
res_lst = list(data_df["ScreenResolution"])

resolutions = []
for item in res_lst:
    match = re.search(r'(\d+x\d+)', item)
    if match:
        resolutions.append(match.group(0))

data_df["Resolution"] = pd.Series(resolutions)
data_df["Width"] = data_df["Resolution"].str.split("x").str[0].astype(int)
# data_df["Height"] = data_df["Resolution"].str.split("x").str[1].astype(int)
data_df["Cpu_Provider"] = data_df["Cpu"].str.split(" ").str[0]
# data_df["Gpu_Provider"] = data_df["Gpu"].str.split(" ").str[0]
data_df["GHz"]  = data_df["Cpu"].str.split(" ").str[-1].str.replace("GHz", "").astype(float)
# data_df["Weight"] = data_df["Weight"].str.replace("kg", "").astype(float)
data_df["Memory"].unique()
memory_lst = list(data_df["Memory"].str.split(" ").str[0])

memory = []
for item in memory_lst:
    match = re.search(r'(\d+)', item)
    if match:
        memory.append(float(match.group(0)))
data_df["Memory_1"] = pd.Series(memory)
data_df["Memory_1"].unique()
memory_lst = list(data_df["Memory"].str.split(" + ").str[1])
memory = []
for item in memory_lst:
    try:
        match = re.search(r'(\d+)', item)
        if match:
            memory.append(int(str(match.group(0))))
    except:
        memory.append(0)
# data_df["Memory_2"] = pd.Series(memory)
tb = (data_df["Memory_1"] < 5) & (0 < data_df["Memory_1"])
data_df.loc[tb,"Memory_1"] *= 1000
data_df.loc[tb]
# tb = (data_df["Memory_2"] < 5) & (0 < data_df["Memory_2"])
# data_df.loc[tb,"Memory_2"] *= 1000
# data_df["Storage_1"] = data_df["Memory"].str.split(" + ").str[0].str.split(" ").str[1]
data_df["Storage_2"] = data_df["Memory"].str.split(" + ").str[1].str.split(" ").str[1].fillna(0)
data_df["Ram"] = data_df["Ram"].str.replace("GB", "").astype(float)
data_df["OpSys"].replace('Mac OS X', 'macOS X', inplace=True)
data_df["OpSys_provider"] = data_df["OpSys"].str.split(" ").str[0]

# ML

ml_data_df = data_df.copy()
ml_data_df.drop(["ScreenResolution", 
                "Memory",
                "Cpu",
                "Memory",
                "Gpu",
                "Resolution",
                # "Height"
                ], 
                axis=1, inplace=True)

cat_lst = ["TypeName", 
           "Company", 
           "OpSys", 
           "Cpu_Provider", 
        #    "Gpu_Provider", 
        #    "Storage_1", 
           "Storage_2", 
           "OpSys_provider"]

for cat in cat_lst:
    company = list(enumerate(ml_data_df[cat].unique().tolist()))
    company_dict = {company[i][1] : company[i][0] for i in range(len(company))}
    ml_data_df[cat] = ml_data_df[cat].map(company_dict)

In [19]:
ml_data_df.head()

,Company,TypeName,Inches,Ram,OpSys,Price_euros,Width,Cpu_Provider,GHz,Memory_1,Storage_2,OpSys_provider
0,0,0,15.6,4.0,0,387.0,1366,0,2.2,500.0,0,0
1,1,1,13.3,8.0,0,1379.0,3200,1,2.4,256.0,0,0
2,0,0,17.3,8.0,0,854.0,1920,1,1.6,256.0,0,0
3,2,2,12.5,8.0,0,1483.0,1920,1,2.7,512.0,0,0
4,3,0,17.3,8.0,0,519.0,1600,0,2.5,1000.0,0,0


In [20]:
train = ml_data_df.copy()

-----------------------------------------------------------------------------------------------------------------

## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

In [21]:
X = train.drop(['Price_euros'], axis=1)
y = train['Price_euros'].copy()
X.shape

(912, 11)

In [22]:
y.shape

(912,)

### 2. Dividir X_train, X_test, y_train, y_test

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, shuffle=False)

### 3. Asignar el modelo (vacío) a una variable


In [24]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

lm = LinearRegression() 
lm.fit(X_train, y_train)

predictions = lm.predict(X_test)
# print(predictions)

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

MAE: 266.16465959362455
MSE: 110385.35024797906
RMSE: 332.2429084991568


In [25]:
metrics.r2_score(y_test, predictions)

0.8136439829290595

#### 3.1 Standarizar los datos

In [26]:
from sklearn import preprocessing

std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_scal = std_scale.transform(X_train)
X_test_scal = std_scale.transform(X_test)


In [27]:
lm_scal = LinearRegression()
lm_scal.fit(X_train_scal, y_train)

predictions = lm_scal.predict(X_test_scal)
# print(predictions)

intercept_scal = lm_scal.intercept_
features_std = pd.DataFrame(lm_scal.coef_, X_train.columns, columns=['coefficient'])

features_std.sort_values('coefficient', ascending=False)

,coefficient
Ram,335.729984
Width,156.739492
GHz,129.989227
OpSys,118.385294
TypeName,102.336739
Cpu_Provider,74.247324
Company,36.555761
Inches,-19.252775
Storage_2,-41.756788
Memory_1,-67.820976


In [28]:
ml_data_df.columns

Index(['Company', 'TypeName', 'Inches', 'Ram', 'OpSys', 'Price_euros', 'Width',
       'Cpu_Provider', 'GHz', 'Memory_1', 'Storage_2', 'OpSys_provider'],
      dtype='object')

In [29]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

MAE: 266.1646595936247
MSE: 110385.35024797916
RMSE: 332.24290849915695


In [ ]:
predictions

866

In [30]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2)
poly_features.fit(X_train)
X_poly = poly_features.transform(X_train)

In [36]:
print(len(X_train), len(y_train))

866 866


In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, X_train, test_size = 0.2, random_state=42)

In [38]:
pol_reg = LinearRegression()
pol_reg.fit(X_train, y_train)

LinearRegression()

In [39]:
predictions = pol_reg.predict(X_test)

In [44]:
pd.DataFrame(pol_reg.predict(X_poly), columns = ['predictions'])

ValueError: Shape of passed values is (866, 11), indices imply (866, 1)

In [43]:
df_preds = pd.DataFrame(pol_reg.predict(X_poly), columns = ['predictions'])
df_preds['real_value'] = pd.Series(y_test)

df_preds

ValueError: Shape of passed values is (866, 11), indices imply (866, 1)

### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(rmse)
print(mae)
print(lm_scal.score(X_test_scal, y_test))
print(lm_scal.coef_)

In [ ]:
y_test.shape

-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `test.csv` para predecir.

In [ ]:
X_pred = pd.read_csv("data/test.csv")
X_pred.head()

In [ ]:
X_pred.shape

In [ ]:
data_df = X_pred.copy()
data_df.drop(["laptop_ID", "Product", "Weight"], axis=1, inplace=True)
res_lst = list(data_df["ScreenResolution"])

resolutions = []
for item in res_lst:
    match = re.search(r'(\d+x\d+)', item)
    if match:
        resolutions.append(match.group(0))

data_df["Resolution"] = pd.Series(resolutions)
data_df["Width"] = data_df["Resolution"].str.split("x").str[0].astype(int)
# data_df["Height"] = data_df["Resolution"].str.split("x").str[1].astype(int)
data_df["Cpu_Provider"] = data_df["Cpu"].str.split(" ").str[0]
# data_df["Gpu_Provider"] = data_df["Gpu"].str.split(" ").str[0]
data_df["GHz"]  = data_df["Cpu"].str.split(" ").str[-1].str.replace("GHz", "").astype(float)
# data_df["Weight"] = data_df["Weight"].str.replace("kg", "").astype(float)
data_df["Memory"].unique()
memory_lst = list(data_df["Memory"].str.split(" ").str[0])

memory = []
for item in memory_lst:
    match = re.search(r'(\d+)', item)
    if match:
        memory.append(float(match.group(0)))
data_df["Memory_1"] = pd.Series(memory)
data_df["Memory_1"].unique()
memory_lst = list(data_df["Memory"].str.split(" + ").str[1])
memory = []
for item in memory_lst:
    try:
        match = re.search(r'(\d+)', item)
        if match:
            memory.append(int(str(match.group(0))))
    except:
        memory.append(0)
data_df["Memory_2"] = pd.Series(memory)
tb = (data_df["Memory_1"] < 5) & (0 < data_df["Memory_1"])
data_df.loc[tb,"Memory_1"] *= 1000
data_df.loc[tb]
tb = (data_df["Memory_2"] < 5) & (0 < data_df["Memory_2"])
data_df.loc[tb,"Memory_2"] *= 1000
# data_df["Storage_1"] = data_df["Memory"].str.split(" + ").str[0].str.split(" ").str[1]
data_df["Storage_2"] = data_df["Memory"].str.split(" + ").str[1].str.split(" ").str[1].fillna(0)
data_df["Ram"] = data_df["Ram"].str.replace("GB", "").astype(float)
data_df["OpSys"].replace('Mac OS X', 'macOS X', inplace=True)
data_df["OpSys_provider"] = data_df["OpSys"].str.split(" ").str[0]

# ML

ml_data_df = data_df.copy()
ml_data_df.drop(["ScreenResolution", 
                "Memory",
                "Cpu",
                "Memory",
                "Gpu",
                "Resolution",
                # "Height"
                ], 
                axis=1, inplace=True)

cat_lst = ["TypeName", 
           "Company", 
           "OpSys", 
           "Cpu_Provider", 
        #    "Gpu_Provider", 
        #    "Storage_1", 
           "Storage_2", 
           "OpSys_provider"]

for cat in cat_lst:
    company = list(enumerate(ml_data_df[cat].unique().tolist()))
    company_dict = {company[i][1] : company[i][0] for i in range(len(company))}
    ml_data_df[cat] = ml_data_df[cat].map(company_dict)

In [ ]:
ml_data_df.info()

In [ ]:
X_pred = ml_data_df.copy()

In [ ]:
len(X_pred.columns)

In [ ]:
from sklearn import preprocessing

std_scale = preprocessing.StandardScaler().fit(X_pred)
X_pred_scal = std_scale.transform(X_pred)

predictions = lm_scal.predict(X_pred_scal)

intercept_scal = lm_scal.intercept_
features_std = pd.DataFrame(lm_scal.coef_, X_pred.columns, columns=['coefficient'])

features_std.sort_values('coefficient', ascending=False)

In [ ]:
pred_df = pd.Series(predictions)

In [ ]:
sample = pd.read_csv("data/sample_submission.csv")

In [ ]:
submission = pd.DataFrame({"laptop_ID": sample["laptop_ID"], "Price_euros": pred_df})

In [ ]:
submission

IMPORTANTE: APLICAR LO MISMO A ESTOS DATOS QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [ ]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

In [ ]:
X_pred = pd.read_csv("Dataset/test.csv", index_col=0)
X_pred.head()

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [ ]:
predictions_submit = model.predict(X_pred)
predictions_submit

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [ ]:
sample = pd.read_csv("Dataset/sample_submission.csv")

In [ ]:
sample.head()

In [ ]:
sample.shape

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [ ]:
submission = pd.DataFrame({"laptop_ID": sample["laptop_ID"], "Price_euros": predictions_submit})

In [ ]:
submission.head()

In [ ]:
submission.shape

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [ ]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission_1.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [ ]:
chequeator(submission)